# Generating Simple Audio Signals

[back to main page](index.ipynb)

We create a simple sine wave as an introductory example using Python and NumPy.

Let's start with some basic Python. We create three numbers and give each of them a name.

> Sometimes these are called *variables*, but that is actually misleading.
> Think about objects and names. An object is created (in our case a number) and we specify a name by which we want to access this object.

In [ ]:
dur = 1.5  # duration in seconds
amp = 0.3  # amplitude (full scale: +-1.0)
freq = 440.  # frequency in Hertz
fs = 44100  # sampling frequency in Hertz

Everything following a `#` sign is a comment.

There is no output. If you want to see which object a certain name refers to, just type the name:

In [ ]:
dur

In Python, *everything* is an object. And every object has a *type*. Let's see ...

In [ ]:
type(dur), type(amp), type(freq), type(fs)

Note that we didn't specify any types explicitly, we just got Python's default types. Numbers with a comma are by default of type `float`, numbers without comma are of type `int`.

There are many more types in Python (strings, tuples, lists, dictionaries, ...), but let's ignore them for now.

If you want an overview about all the objects you have defined up to now, use `%who` or its more verbose cousin `%whos`.

In [ ]:
%whos

Now, let's bring NumPy into the game. The canonical way to do that is

In [ ]:
import numpy as np

With this, we'll have to prepend "`np.`" to all NumPy functions, types etc.

Now let's create the most basic signal, a sine. This is kind of the "Hello, world!" of signals.

In order to create a sine tone, we need first a series of time instances to represent our sampling times. The distance between those instances is the *sampling interval* $\tau = \frac{1}{f_s}$, where $f_s$ is the *sampling frequency*.

To create a series of regularly ascending (or descending) values, NumPy provides the function `arange()`. Let's use that.

In [ ]:
t = np.arange(0, np.ceil(dur * fs)) / fs
t

As you can see, this creates an array of numbers from 0 to just below the value specified in `dur`. These are a lot of numbers, so to avoid flooding us with heaps of useless output, IPython just shows the first and last few values. Very handy.

> Note that in Python 2 and before, the division operator works a little differently then in Python 3. Here we assume we're using the latter, where the division of two `int`s returns a `float` value (and not an `int` with the truncated result!).
> To make sure this also works as expected in Python 2, you can convert one operand to `float` before the division or use a special `import` statement:
>
> ```python
> 1. / fs
> ```
>
> or
>
> ```python
> 1 / float(fs)
> ```
>
> or
>
> ```python
> from __future__ import division
> 1 / fs
> ```

If you want to get help about `np.arange()` (or any other function/object), just write its name followed (or preceded) by a question mark:

In [ ]:
np.arange?

This will open a help screen at the bottom of your browser window with an explanation of all parameters and with a few usage examples.

Now let's check the type of `t`.

In [ ]:
type(t)

The `ndarray` is the single most important type of NumPy. It can handle arrays with an arbitrary number of dimensions. All values stored in an `ndarray` have the same data type. This makes most operations on them faster then on Python's built-in `list`s.

Let's get some information about our brand new array named `t`.

In [ ]:
len(t), t.dtype, t.ndim

By default, NumPy stores floating point numbers as `float64`, i.e. each number using 64 bits. This is sometimes called *double precision*. If you want *single precision*, you can use `float32`. The length of our array can be obtained with the built-in `len()` function and `t.ndim` shows how many dimensions the array has. This one has only one. Boring.

Now that we have our time instances, we can compute the sine for each of them, according to the equation

$$x(t) = A\sin(\omega t),$$

where $A$ is the amplitude, $\omega = 2\pi f$ and $f$ is the desired frequency of the resulting sine tone.

In [ ]:
sig = amp * np.sin(2 * np.pi * freq * t)
sig

Note that we didn't have to explicitly loop over the array elements. Most NumPy functions - including `np.sin()` - work element-wise. If an array is multiplied by a scalar, the multiplication is also applied element-wise.

> This is called "broadcasting", but more about that later ...

To check if this actually worked, we plot the signal, but first we'll set up inline plotting.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.plot(sig)
plt.ylim(-1.1, 1.1);

Hmmm ... maybe we recognize something if we only plot the first 200 values.

In [ ]:
plt.plot(sig[:200])
plt.xlabel("time / samples")
plt.ylim(-1.1, 1.1);

Better.

If we want to show the time in milliseconds instead of samples, we have to specify the time instances:

In [ ]:
plt.plot(t[:200] * 1000, sig[:200])
plt.title("sine tone")
plt.xlabel("time / milliseconds")
plt.ylim(-1.1, 1.1);

Now that we can *see* our signal, wouldn't it also be nice to hear it?

There are several 3rd-party libraries for Python that allow audio playback, let's try one of them: [python-sounddevice](http://python-sounddevice.rtfd.org/).

Follow the [installation instructions](http://python-sounddevice.rtfd.org/#Installation), then re-evaluate this notebook (with the menu command "Cell" $\to$ "Run All Above") and continue here:

In [ ]:
import sounddevice as sd

In [ ]:
sd.play(sig, fs)

In [ ]:
# TODO: save WAV file, link to sudio-file-notebook?

OK, now we know how to create a *mono* signal, but what about *stereo*?

For now, we were using one-dimensional arrays, but for stereo signals we need two-dimensional arrays.
The 2 channels can be either stored as rows or as columns of such a two-dimensional array.
Both ways are possible, but it's more common to store the channels as columns.

Let's create two sine signals with different frequencies, one on the left channel and one on the right.

There are several possibilities:

1. Create two separate mono signals like above, and combine them with numpy.column_stack()

1. Duplicate the time array `t` and the array with the two frequencies with numpy.tile() and use those larger two-dimensional arrays in the calculations (this is not recommended)

1. Use *broadcasting* and do everything at once

1. For sure, there are more ways to do this ...

In this case, let's use *broadcasting*!

This means that we use arrays of different (but compatible) shapes in our calculations and the singular dimensions are internally repeated.
This sounds complicated, let's just see it in an example.

Remember our time array `t`?

In [ ]:
t

This array is one-dimensional (note that there is only one pair of brackets).

But we can re-shape it into a two-dimensional array with one column.
We can specify the concrete shape, or we can set one component to `-1`, which means this dimension is determined automatically, based on the given data.

In [ ]:
t.reshape(-1, 1)

The result is two-dimensional (note the two levels of brackets!).

Let's now create an array for our two frequencies.

In [ ]:
freq = np.array([500, 600])
freq

As you can see, this two-element array is one-dimensional.

The interesting part comes when we multiply those two arrays:

In [ ]:
freq * t.reshape(-1, 1)

Wow, the result is a two-dimensional array with two columns!
The left column holds the time array element-wise multiplied with the left frequency, the right column used the right frequency value for multiplication.

Isn't this great?
Now we can use this for creating our *stereo* sine signal:

In [ ]:
sig = amp * np.sin(2 * np.pi * freq * t.reshape(-1, 1))
sig

Again, we have an array with two columns, one for each channel.

The plt.plot() function can handle two-dimensional arrays and just plots each column with a new color.

In [ ]:
plt.plot(sig[:200])
plt.ylim(-1.1, 1.1);

Of course we can again plot the time in milliseconds.
And let's add a legend in order to distinguish the two signals!

In [ ]:
plt.plot(t[:200] * 1000, sig[:200])
plt.title("stereo sine")
plt.xlabel("time / milliseconds")
plt.legend(["left", "right"])
plt.ylim(-1.1, 1.1);

In [ ]:
# TODO: link to plotting intro

In [ ]:
sd.play(sig, fs)

<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <span resource="[_:publisher]" rel="dct:publisher">
    <span property="dct:title">Matthias Geier</span></span>
  has waived all copyright and related or neighboring rights to
  this work.
</p>